# Data Collection and Import

In [2]:
# accessing APIs and URLs
import requests

# static web scraping
from urllib.request import urlopen
from lxml.html import parse, fromstring

# regular expressions
import re

# downloading files
import urllib.request

# operating system
import os

# looping through folder
import glob

# read pdfs
import fitz 

# NLP
from nltk.corpus import PlaintextCorpusReader

In [3]:
# extract all links from website
tree=parse(urlopen("https://www.americanrhetoric.com/barackobamaspeeches.htm"))
linkelements = tree.xpath("//a")
links = [e.attrib["href"] for e in linkelements]

In [4]:
len(links)

1417

In [5]:
# only retain pdf links
p = re.compile('.*pdf$')
pdf_links = [ s for s in links if p.match(s) ]

In [6]:
len(pdf_links)

436

In [7]:
# concetenate baseurl and path
baseurl="https://www.americanrhetoric.com/"
full_pdf_links = [baseurl + link for link in pdf_links]

In [8]:
# create new folder
os.makedirs("data")

In [9]:
# download all pdf files
i = 1
for link in full_pdf_links:
    x = "data/file_" + str(i) + ".pdf"
    urllib.request.urlretrieve(link, x)
    i = i+1

In [43]:
# sort pdf files by name
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

filename_list = sorted(glob.glob("data/*.pdf"), key=numericalSort)

# create new folder
os.makedirs("data_txt")

# loop through all files and save as txt filles in data_txt folder
for filename in filename_list:
    with fitz.open(filename) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    name = filename.replace("pdf", "txt")
    name = name.replace("data", "data_txt")
    f = open(name, "a")
    f.write(text)
    f.close()

In [84]:
# import corpus
corpus_root = 'data_txt/'
speeches = PlaintextCorpusReader(corpus_root, '/file.*\.txt')

In [85]:
# print first speech
speeches.raw('file_1.txt')

'  \nAAm\nmeerriiccaannRRhheettoorriicc..ccoom\nm  \n \nAmericanRhetoric.com \nUpdated 12/7/21 \n     Page 1 \nBarack Obama \n2004 Democratic National Convention Keynote Address \ndelivered 27 July 2004, Fleet Center, Boston \n \n \nAUTHENTICITY CERTIFIED: Text version below transcribed directly from audio \nOn behalf of the great state of Illinois, crossroads of a nation, Land of Lincoln, let me express \nmy deepest gratitude for the privilege of addressing this convention. \nTonight is a particular honor for me because, let’s face it, my presence on this stage is pretty \nunlikely. My father was a foreign student, born and raised in a small village in Kenya. He grew \nup herding goats, went to school in a tin-roof shack. His father -- my grandfather -- was a \ncook, a domestic servant to the British. \nBut my grandfather had larger dreams for his son. Through hard work and perseverance my \nfather got a scholarship to study in a magical place, America, that shone as a beacon of \nfre